In [14]:
from langchain import LLMChain, PromptTemplate
import openai, langchain
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.simplefilter('ignore')

In [15]:
load_dotenv(find_dotenv('newenv.env'))

True

In [16]:
langchain.__version__

'0.2.13'

In [17]:
openai.__version__

'1.40.6'

In [27]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
print(completion.choices[0].message.content)

In the land of code where logic reigns,  
Where loops abound and flow charts wane,  
There's a tale of a curious function,  
A magical trick called recursion.

Picture a mirror in a hall so wide,  
Reflecting reflections, on either side.  
A call to the self, a loop so divine,  
Where each answer awaits, as it starts to unwind.

“A problem, oh problem!” begins with despair,  
But the recursive function whispers with flair,  
“Break it down gently, into smaller parts,  
For within each piece, the answer imparts.”

First, check for a base case, steady and sure,  
Without it, dear coder, you’ll find no cure.  
It’s the ending condition, the stop signal’s shout,  
A lighthouse in fog, guiding you out.

Then comes the call, like a ripple on streams,  
“Here’s a smaller version,” the function redeems.  
It plunges through layers, like diving in deep,  
With each recursive call, you take a new leap.

As the layers stack up, you climb to the peak,  
Where solutions lie waiting, not far, just a

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
messages= [
    ("system", "You are an expert assistant about movies."),
    ("human", "Can you tell top 5 movies of all time by revenue?")
]

print(llm.invoke(messages).content)

As of October 2023, the top five highest-grossing movies of all time by worldwide box office revenue are:

1. **Avatar** (2009) - Directed by James Cameron, this sci-fi epic has grossed over $2.9 billion.
2. **Avengers: Endgame** (2019) - Directed by Anthony and Joe Russo, this Marvel superhero film has grossed approximately $2.798 billion.
3. **Titanic** (1997) - Also directed by James Cameron, this romantic drama has grossed around $2.2 billion.
4. **Star Wars: The Force Awakens** (2015) - Directed by J.J. Abrams, this installment of the Star Wars saga has grossed about $2.068 billion.
5. **Avengers: Infinity War** (2018) - Directed by Anthony and Joe Russo, this film has grossed around $2.048 billion.

Please note that box office figures are constantly changing due to re-releases and new films, so it's always good to check for the most current data.


In [20]:
# from langchain.llms import OpenAI
# llm = OpenAI(model = "gpt-3.5-turbo-instruct", temperature=0) #gpt-4o-mini, gpt-3.5-turbo, gpt-4o are chat models so use ChatOpenAI for those

In [23]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Set the query you want to ask
input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

# Run the LLMChain to get the AI-generated answer
response = chain.run(input_data)

print("Question:", input_data["query"])
print("Answer:", response)

Question: What is the main advantage of quantum computing over classical computing?
Answer: The main advantage of quantum computing over classical computing is its ability to solve complex problems faster due to its use of quantum mechanics.


To create a PromptTemplate object, two arguments are required:

- input_variables: A list of variable names in the template; in this case, it includes only the query.
- template: The template string containing formatted text and placeholders.

After creating the PromptTemplate object, it can be used to produce prompts with specific questions by providing input data. The input data is a dictionary where the key corresponds to the variable name in the template. The resulting prompt can then be passed to a language model to generate answers.- 

# FewShotPromptTemplate

In [24]:
from langchain import FewShotPromptTemplate

In [25]:
examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

chain = LLMChain(llm=llm, prompt=dynamic_prompt)

input_data = {"input": "tiger"}
response = chain.run(input_data)

print(response)

Animal: tiger  
Habitat: tropical rainforests, grasslands, and mangrove swamps


Additionally, you can also save your PromptTemplate to a file in your local filesystem in JSON or YAML format:

     prompt_template.save("awesome_prompt.json")

and load it back 

    from langchain.prompts import load_prompt
    loaded_prompt = load_prompt("awesome_prompt.json")

In [26]:
#sarcasm example

examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "How can I learn quantum computing?"}
response = chain.run(input_data)

print(response)

Start by mastering the art of confusing your friends with Schrödinger jokes. Then, dive into some textbooks that make you question your sanity!
